# Übungsblatt 6

**Lernziele**

In den Übungen dieser Wochen lernen Sie:

* Funktionen der Standardbibliothek zum Umgang mit Listen
* Records
* Aufzählungstypen

## Aufgabe 6.1 (P) Java und OCaml: Generics und Polymorphie, Wert- und Referenzgleichheit, Namespacing

Ähnlich wie wir mit Funktionen über Werte abstrahieren können, geht dies auch für Typen.
Es lassen sich Funktionen definieren, die auf beliebigen Typen funktionieren, genauso wie sich generische Typen mit Typvariablen definieren lassen.
In Java heißt dieses Feature Generics, im Kontext von funktionalen Sprachen spricht man von Polymorphie und abstrakten Datentypen.

Geben Sie eine entsprechende OCaml-Implementierung für den folgenden Java-Code.

```java
class Pair<L,R> {
    L left;
    R right;

    public Pair(L left, R right) {
        this.left = left;
        this.right = right;
    }

    @Override
    public boolean equals(Object o) {
        if (!(o instanceof Pair)) return false;
        Pair pair = (Pair) o;
        return this.left.equals(pair.left) &&
            this.right.equals(pair.right);
    }

    static <L, R> Pair<R, L> flip(Pair<L, R> pair) {
        return new Pair<R, L>(pair.right, pair.left);
    }
}

public class Generics {
    public static void main(String[] args) {
        Pair<Integer, String> a = new Pair<>(1, "one");
        Pair<Integer, String> b = new Pair<>(1, "one");
        System.out.println(a == b); // Referenzgleichheit: false
        System.out.println(a.equals(b)); // Wertgleichheit: true
        System.out.println(Pair.flip(a).left); // "one"
    }
}
```

In [ ]:
(* TODO *)

## Aufgabe 6.2 (P) Exceptions keine Option?

Viele Funktionen der Standardbibliothek wie z.B. `List.find : ('a -> bool) -> 'a list -> 'a` werfen eine Exception (in diesem Fall z.B. `Not_found` bei `List.find ((<) 2) [1;2]`. Warum ist dies gefährlich?

In [ ]:
List.find

In [ ]:
(<)

In [ ]:
(<) 2

In [ ]:
List.find ((<) 2) [1;2]

Operatoren sind auch nur normale Funktionen, die neu definiert werden können.

In [ ]:
let (+) = (-) in 3 + 2

In [ ]:
let (-!>) a b = a - 2 * b in 2 -!> 3

In unseren Implementierungen wollen wir deshalb stattdessen den Typ `'a option` benutzen. 
Im Interpreter (z.B. utop) können Sie sich Definitionen verschiedener Identifier ausgeben lassen.
So liefert uns `#show_type option;;`:

In [ ]:
#show_type option

*Bemerkung*: Wert-Definitionen sind standardmäßig nicht-rekursiv, Typ-Definitionen hingegen standardmäßig rekursiv (siehe `'a tree` weiter unten).
Das `nonrec` ist hier aber nicht relevant (rechts kein ungebundener Typ).

Da es mühsam wäre jedes Mal ein Pattern-Matching auf Option-Werte zu machen,
definieren Sie eine Funktion `(|?) : 'a option -> 'a -> 'a` welche im Falle von `None` den rechten Wert liefert.

In [ ]:
let (!?) a b = ...

Zum Beispiel soll folgender Ausdruck für `[1;2;3;4]` den Wert `2` und für `[1;3]` den Wert `0` liefern.

In [ ]:
let f xs = List.find_opt (fun x -> x mod 2 = 0) xs |? 0

Ein weiterer hilfreicher Operator ist `>>=` (wird oft "bind" genannt).

In [ ]:
let (>>=) a f = match a with Some x -> f x | None -> None

## Aufgabe 6.3 (P) Binärbaum

Gegeben sei folgende Typ-Definition eines binären Baumes:

In [ ]:
type 'a tree = Node of 'a * 'a tree * 'a tree | Leaf

Implementieren Sie die Funktion

```ocaml
val insert : 'a -> 'a tree -> 'a tree
```

die einen Wert in den Baum einfügt und den neuen Baum zurückliefert. Beim Einfügen
in den Suchbaum sollen kleinere Elemente links im Baum zu finden sein.

In [ ]:
(* TODO *)

Implementieren Sie die Funktion

```ocaml
val height : 'a tree -> int
```

die die Höhe des Baumes berechnet. Ein Blatt hat dabei die Höhe 0.

In [ ]:
(* TODO *)

Implementieren Sie die Funktionen

```ocaml
val min_elem : 'a tree -> 'a option
```

und

```ocaml
val max_elem : 'a tree -> 'a option
```

die das kleinste bzw. größte Element eines Baumes zurückliefern, sofern ein solches existiert.

In [ ]:
(* TODO *)

Implementieren Sie die Funktion

```ocaml
val remove : 'a -> 'a tree -> 'a tree
```

die ein Element aus dem Baum entfernt und den neuen Baum zurückliefert.

In [ ]:
(* TODO *)